## Initial Stuff

### Connect to Drive

In [1]:
connect_to_drive = False

In [2]:
#Run command and authorize by popup --> other window
if connect_to_drive:
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)

### Install packages

In [3]:
if connect_to_drive:
    #Install FS code
    !pip install  --upgrade --force-reinstall git+https://github.com/federicosiciliano/easy_lightning.git

    !pip install pytorch_lightning

### IMPORTS

In [4]:
#Put all imports here
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
#import pickle
import os
import sys
#import cv2
import csv
import torch

### Define paths

In [5]:
#every path should start from the project folder:
project_folder = "../"
if connect_to_drive:
    project_folder = "/content/gdrive/Shareddrives/<SharedDriveName>" #Name of SharedDrive folder
    #project_folder = "/content/gdrive/MyDrive/<MyDriveName>" #Name of MyDrive folder

#Config folder should contain hyperparameters configurations
cfg_folder = os.path.join(project_folder,"cfg")

#Data folder should contain raw and preprocessed data
data_folder = os.path.join(project_folder,"data")
raw_data_folder = os.path.join(data_folder,"raw")
processed_data_folder = os.path.join(data_folder,"processed")

#Source folder should contain all the (essential) source code
source_folder = os.path.join(project_folder,"src")

#The out folder should contain all outputs: models, results, plots, etc.
out_folder = os.path.join(project_folder,"out")
img_folder = os.path.join(out_folder,"img")

### Import own code

In [6]:
#To import from src:

#attach the source folder to the start of sys.path
sys.path.insert(0, project_folder)

#import from src directory
# from src import ??? as additional_module
import easy_rec as additional_module #REMOVE THIS LINE IF IMPORTING OWN ADDITIONAL MODULE

import easy_exp, easy_rec, easy_torch #easy_data

/home/filippobetello/.conda/envs/recsys/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-17 14:36:11,014	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-17 14:36:11,074	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


### Create Data

In [7]:
cfg = easy_exp.cfg.load_configuration("config_rec")

In [8]:
#cfg["data_params"]["test_sizes"] = [cfg["data_params.dataset_params.out_seq_len.val"],cfg["data_params.dataset_params.out_seq_len.test"]]

data_params = cfg["data_params"].copy()
data_params["data_folder"] = raw_data_folder

data, maps = easy_rec.data_generation_utils.preprocess_dataset(**data_params)

Ratings data already exists. Skip pre-processing
Filtering by minimum number of users per item: 5
Filtering by minimum number of items per user: 5
Densifying index
Splitting: leave_n_out


In [9]:
item_infos = easy_rec.data_generation_utils.load_item_info('../data/raw', "ml-100k")
print(len(item_infos))
print(item_infos.keys())
print(item_infos[['movie_id', 'movie_title']].head(3))

1682
Index(['movie_id', 'movie_title', 'release date', 'video release date',
       'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children's',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'],
      dtype='object')
   movie_id        movie_title
0         1   Toy Story (1995)
1         2   GoldenEye (1995)
2         3  Four Rooms (1995)


In [10]:
data['train_sid']

array([list([167, 171, 164, 155, 165, 195, 186, 249, 126, 14, 117, 109, 180, 1, 245, 247, 256, 50, 248, 252, 261, 93, 223, 123, 19, 136, 122, 145, 7, 234, 15, 244, 24, 263, 125, 259, 236, 13, 25, 120, 250, 235, 239, 118, 129, 47, 65, 189, 31, 28, 39, 114, 52, 237, 69, 182, 11, 198, 95, 160, 178, 60, 83, 134, 64, 98, 22, 26, 162, 201, 8, 89, 213, 181, 48, 99, 174, 191, 159, 127, 179, 184, 142, 68, 55, 203, 56, 96, 79, 81, 150, 211, 23, 70, 84, 196, 190, 94, 183, 133, 206, 144, 187, 185, 97, 36, 158, 85, 143, 17, 173, 251, 105, 147, 108, 146, 219, 242, 106, 121, 104, 246, 107, 119, 45, 267, 266, 258, 260, 262, 10, 149, 233, 71, 42, 92, 175, 91, 192, 216, 176, 215, 193, 73, 59, 41, 132, 194, 169, 217, 212, 156, 222, 226, 27, 80, 230, 67, 199, 4, 214, 163, 2, 205, 77, 135, 53, 46, 40, 152, 268, 253, 115, 172, 210, 228, 154, 202, 62, 90, 218, 166, 161, 61, 229, 35, 112, 264, 57, 49, 30, 232, 130, 151, 82, 140, 72, 157, 33, 197, 113, 224, 21, 88, 148, 238, 110, 101, 103, 43, 34, 29, 131, 204

In [11]:
item_infos['movie_title'].values

array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
       'Sliding Doors (1998)', 'You So Crazy (1994)',
       'Scream of Stone (Schrei aus Stein) (1991)'],
      shape=(1682,), dtype=object)

## Embedding computation


In [52]:
RUN ONLY IF NECESSARY

SyntaxError: invalid syntax (3949450103.py, line 1)

In [12]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')


In [13]:
import pandas as pd
# Get all movie titles
titles = item_infos['movie_title'].tolist()

# Compute embeddings
print("Computing embeddings...")
embeddings = model.encode(titles, show_progress_bar=True)

# Create a DataFrame with movie IDs, titles, and their embeddings
embeddings_df = pd.DataFrame({
    'movie_id': item_infos['movie_id'],
    'title': item_infos['movie_title'],
    'embedding': list(embeddings)
})

print("\nShape of embeddings:", embeddings.shape)
print("\nFirst movie title and its embedding shape:")
print(f"Title: {embeddings_df['title'].iloc[0]}")
print(f"Embedding shape: {len(embeddings_df['embedding'].iloc[0])}")

Computing embeddings...


Batches: 100%|██████████| 53/53 [00:01<00:00, 42.65it/s]


Shape of embeddings: (1682, 384)

First movie title and its embedding shape:
Title: Toy Story (1995)
Embedding shape: 384


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity between one movie and all others in list of lists data['train_sid']
# for each user save the corresponding movie title : movie similar with a cosine similarity threshold of 0.5

d = {}
counter = 1
for user_list in data['train_sid']:
    user_key = str(counter)
    d[user_key] = {}
    counter+=1

    for movie_id in range(len(user_list)):

        movie_embedding = embeddings_df[embeddings_df['movie_id'] == user_list[movie_id]]['embedding'].values[0]

        similar_movies = []
        
        for j in range(movie_id, len(user_list)): 
            comparison_movie_id = user_list[j]
            comparison_embedding = embeddings_df.loc[embeddings_df['movie_id'] == comparison_movie_id, 'embedding'].values[0]
            
            similarity = cosine_similarity([movie_embedding], [comparison_embedding])[0][0]
            
            if similarity > 0.75 and similarity < 0.98:  # so we avoid to save the same title
                movie_title = embeddings_df.loc[embeddings_df['movie_id'] == comparison_movie_id, 'title'].values[0]
                similar_movies.append(movie_title)
        if similar_movies != []:
            movie_title = embeddings_df.loc[embeddings_df['movie_id'] == user_list[movie_id], 'title'].values[0]
            d[user_key][movie_title] = similar_movies

# save the dictionary into a json file
import json
with open('similar_movies.json', 'w') as f:
    json.dump(d, f)




## Modification of the data

In [12]:
import json


with open('similar_movies_07.json') as f:
    similar_movies = json.load(f)

In [13]:
similar_movies['1']

{'Terminator, The (1984)': ['Terminator 2: Judgment Day (1991)'],
 'Lost World: Jurassic Park, The (1997)': ['Jurassic Park (1993)'],
 'George of the Jungle (1997)': ['Jungle2Jungle (1997)'],
 'Three Colors: Blue (1993)': ['Three Colors: Red (1994)',
  'Three Colors: White (1994)'],
 'Godfather, The (1972)': ['Godfather: Part II, The (1974)'],
 'Alien (1979)': ['Aliens (1986)'],
 'Die Hard (1988)': ['Die Hard 2 (1990)'],
 'From Dusk Till Dawn (1996)': ['Big Night (1996)'],
 'Three Colors: Red (1994)': ['Three Colors: White (1994)'],
 'Top Gun (1986)': ['Young Guns (1988)'],
 'Free Willy 2: The Adventure Home (1995)': ['Free Willy (1993)'],
 'Batman Forever (1995)': ['Batman Returns (1992)', 'Batman & Robin (1997)']}

In [14]:
new_similar_movies = {}

for i in similar_movies:
    new_similar_movies[str(i)] = {}
    for j in similar_movies[i]:

        # find the movie_id correspondent to the movie title    
        app = []
        for k in similar_movies[i][j]:
            app.append(int(item_infos.loc[item_infos['movie_title'] == k, 'movie_id'].values[0]))
        movie_id = int(item_infos.loc[item_infos['movie_title'] == j, 'movie_id'].values[0])
        new_similar_movies[str(i)][movie_id] = app
        


In [15]:
app = 0
for i in data['train_sid']:
    app += len(i)

print(app/len(data['train_sid']))

103.28844114528101


In [16]:
for i in range(len(data['train_sid'])):
    for j in list(new_similar_movies[str(i+1)].keys()):
        try:
            data['train_sid'][i].remove(j)
        except:
            print('error')
            continue


error
error
error
error


In [17]:
app = 0
for i in data['train_sid']:
    app += len(i)

print(app/len(data['train_sid']))

101.06680805938494


## Training model

In [18]:
datasets = easy_rec.rec_torch.prepare_rec_datasets(data,**data_params["dataset_params"])

In [19]:
collator_params = cfg["data_params"]["collator_params"].copy()
collator_params["num_items"] = np.max(list(maps["sid"].values()))

In [20]:
collators = easy_rec.rec_torch.prepare_rec_collators(**collator_params)

In [21]:
loader_params = cfg["model"]["loader_params"].copy()
loaders = easy_rec.rec_torch.prepare_rec_data_loaders(datasets, **loader_params, collate_fn=collators)

In [22]:
rec_model_params = cfg["model"]["rec_model"].copy()
rec_model_params["num_items"] = np.max(list(maps["sid"].values()))
rec_model_params["num_users"] = np.max(list(maps["uid"].values()))
rec_model_params["lookback"] = data_params["collator_params"]["lookback"]

In [23]:
main_module = easy_rec.rec_torch.create_rec_model(**rec_model_params)#, graph=easy_rec.data_generation_utils.get_graph_representation(data["train_sid"]))

Seed set to 42


In [24]:
exp_found, experiment_id = easy_exp.exp.get_set_experiment_id(cfg)
print("Experiment already found:", exp_found, "----> The experiment id is:", experiment_id)

Experiment already found: False ----> The experiment id is: Z1llvrPrwYDCYzTW


In [25]:
model_params = cfg["model"].copy()

trainer_params = easy_torch.preparation.prepare_experiment_id(model_params["trainer_params"], experiment_id)

#dynamic_negatives_index = [i for i, x in enumerate(trainer_params["callbacks"]) if "DynamicNegatives" in x][0]
#trainer_params["callbacks"][dynamic_negatives_index]["DynamicNegatives"]["dataloader"] = loaders["train"]

# Prepare callbacks and logger using the prepared trainer_params
trainer_params["callbacks"] = easy_torch.preparation.prepare_callbacks(trainer_params, additional_module.callbacks)
trainer_params["logger"] = easy_torch.preparation.prepare_logger(trainer_params)

# Prepare the trainer using the prepared trainer_params
trainer = easy_torch.preparation.prepare_trainer(**trainer_params)

model_params["loss"] = easy_torch.preparation.prepare_loss(model_params["loss"], additional_module.losses)

# Prepare the optimizer using configuration from cfg
model_params["optimizer"] = easy_torch.preparation.prepare_optimizer(**model_params["optimizer"])

# Prepare the metrics using configuration from cfg
# num_negatives = {split_name:[x] for split_name,x in data_params["collator_params"]["num_negatives"].items()}
# num_negatives["val"] += num_negatives["test"] #cause using test as val just to get metrics
# model_params["metrics"] = additional_module.metrics.prepare_rank_corrections(model_params["metrics"], num_negatives = num_negatives, num_items = rec_model_params["num_items"])
model_params["metrics"] = easy_torch.preparation.prepare_metrics(model_params["metrics"], additional_module.metrics)

# Create the model using main_module, loss, and optimizer
model = easy_torch.process.create_model(main_module, **model_params)

Seed set to 42
Seed set to 42
Seed set to 42
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42


In [26]:
import time
start = time.time()
easy_torch.process.train_model(trainer, model, loaders, val_key=["val","test"]) #tracker=tracker, profiler=profiler, 
end = time.time()
print("Time elapsed:", end-start)

Seed set to 42
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                        | Params | Mode 
--------------------------------------------------------------------
0 | main_module | GRU4Rec                     | 82.8 K | train
1 | loss        | SequentialBCEWithLogitsLoss | 0      | train
2 | metrics     | RobustModuleDict            | 0      | train
--------------------------------------------------------------------
82.8 K    Trainable params
0         Non-trainable params
82.8 K    Total params
0.331     Total estimated model params size (MB)
52        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/filippobetello/.conda/envs/recsys/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=27` in the `DataLoader` to improve performance.


/home/filippobetello/.conda/envs/recsys/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=27` in the `DataLoader` to improve performance.
/home/filippobetello/.conda/envs/recsys/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (8) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 299: 100%|██████████| 8/8 [00:04<00:00,  1.64it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 299: 100%|██████████| 8/8 [00:04<00:00,  1.63it/s, v_num=0]
Time elapsed: 1479.0283708572388


In [27]:
easy_torch.process.test_model(trainer, model, loaders) #, tracker=tracker, profiler=profiler)

Seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/filippobetello/.conda/envs/recsys/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=27` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 8/8 [00:03<00:00,  2.42it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_MAP_@10        │   0.006522622425109148    │
│       test_MAP_@20        │   0.004005966708064079    │
│        test_MAP_@5        │   0.010169671848416328    │
│       test_MRR_@10        │    0.0222693532705307     │
│       test_MRR_@20        │    0.0222693532705307     │
│        test_MRR_@5        │    0.0222693532705307     │
│       test_NDCG_@10       │    0.0222693532705307     │
│       test_NDCG_@20       │    0.0222693532705307     │
│       test_NDCG_@5        │    0.0222693532705307     │
│    test_Precision_@10     │   0.0022269354667514563   │
│    test_Precision_@20     │   0.0011134677333757281   │
│     test_Precision_@5     │   0.0044538709335029125   │
│      test_Recall_@10      │    0.0222693532705307     │
│      test_Recall_@20      │    0.0222693532705307     │
│      test_Recall_@5       │    0.0222693532705307     │
│         test_loss         │    0.3586817979812622     │
└───────────────────────────┴───────────────────────────┘